In [20]:
import pandas as pd

In [21]:
df = pd.read_pickle('./newsdata/yonhap_all.pkl')

In [22]:
len(df)

836798

In [23]:
df.head(5)

,article,author,date,section,title
0,"보석은 불허…""한 달 이내 새 재판 열릴 것"" 이집트 법원이 1일(현지시간)...",한상용,2015-01-01,국제일반,"이집트 법원, 알자지라 기자 징역형 원심 파기환송(종합)"
1,한 때 브랜드 '빅맨'으로 국내 대표적인 속옷업체로 이름을 날렸던 케이비물...,이대희,2015-01-08,사회일반,'빅맨' 속옷업체 회삿돈 빼돌린 조폭 구속기소
2,미국 캘리포니아 남부에서 동거하던 한국계 남녀가 이동식 주택 안에서 총에 ...,임화섭,2015-01-11,살인,샌디에이고서 한인 남녀 시신 발견…살인후 자살 의심
3,애플이 대화면 아이폰의 인기에 힘입어 삼성전자의 세계 시장 점유율 1위 자...,이윤영,2015-01-27,휴대단말,"""애플, 삼성의 세계판매 1위 자리 바짝 추격""< WSJ>"
4,‘아 그걸 뭐라고 부르지…’외국에서 평범한 생활용품을 사려다 물건 이름이 ...,전승엽,2015-01-28,교육일반,<카드뉴스> 이걸 영어로 뭐라고 할까요


## 섹션 분리

In [24]:
df['sec']=df['section'].apply(lambda x: x.replace('일반', ''))

In [25]:
counts=df.groupby('sec').size()
sorted_counts=counts.sort_values(ascending=False)
sorted_counts

sec
지방행정        111001
사회           99891
국제           84225
정치           82755
경제           80266
기업           18618
산업           17347
문화           14436
중앙행정         14017
외교           12830
국방           11572
재테크          11350
대기업          10513
북한            9373
경찰            8969
IT            8708
교육            8561
국회            7326
소비자           6671
증권시장          6520
금융·증권         5366
편집            5294
대학교육          5009
사건            4642
의료            4500
대통령업무         4419
청와대           4204
중소기업          4151
관광업           3407
지방의회          3336
             ...  
대중가요             4
임대업              4
레이싱              3
주식시황             3
야구               3
해외축구             2
경마               2
해외야구             2
수상레포츠            2
국내대회             2
금시세              2
기업분석             2
배구               2
무속신앙             2
교포               1
익스트림게임           1
승마               1
헤드라인             1
핸드볼              1
OANA             1
유럽연합             1
레슬링     

In [26]:
sorted_counts.to_csv('./yonhap_sec.csv')

In [27]:
df=df.reset_index(drop=True)

In [28]:
len(df)

836798

In [29]:
df_soc=df[df['sec']=='사회']
docs = df_soc.article

In [30]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in t.pos(text) if pos[1] in ['SL','SH','SN','NN','NNG', 'NNP']]
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]


CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 5.65 ms


In [35]:
stopwords_newsis=['지난해', '중','말', '뒤', '곳', '군', '건', '이날', '도', '등', '명', '시', '앞', '원', '분', '회', '년', '것', '씨', '일', '월','오전','오후']

In [36]:
%%time
import numpy as np;
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger,stop_words=stopwords_newsis, ngram_range=(1,3), min_df=2, max_df=.25)
vect_weights = vect.fit_transform(docs)
#sorted(vect.vocabulary_, key=vect.vocabulary_.get, reverse=True)
weights = np.asarray(vect_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vect.get_feature_names(), 'weight': weights})

CPU times: user 8min 35s, sys: 2.44 s, total: 8min 37s
Wall time: 8min 39s


In [33]:
weights_df.sort_values(by='weight', ascending=False).head(50)

,term,weight
160440,경찰,0.012148
2514126,혐의,0.008852
2414342,학교,0.008402
1236199,수,0.008112
1140116,서울,0.007362
1946516,조사,0.007086
449313,김,0.006861
307674,교육,0.006543
2055086,지역,0.006486
1032436,사고,0.006391


## 필요없는 데이터 제거

In [7]:
df.drop(['author', 'date','section'], axis = 1, inplace = True)

## 본문 길이 체크

In [9]:
df['article']=df['article'].apply(lambda x: x.strip())
df['len']=df['article'].apply(lambda x: len(x))
df=df[df['len']>0]

df=df.reset_index(drop=True)

In [11]:
len(df)

3004642

In [10]:
df.tail(4)

,article,title,sec,len
3004638,1일 경상북도 칠곡군 왜관에 위치한 파미힐스CC에서 열린 2017 DGB금융그룹 대...,더 신중하게,스포츠,85
3004639,1일 경상북도 칠곡군 왜관에 위치한 파미힐스CC에서 열린 2017 DGB금융그룹 대...,멀리 멀리,스포츠,85
3004640,1일 경상북도 칠곡군 왜관에 위치한 파미힐스CC에서 열린 2017 DGB금융그룹 대...,어느 위치가 좋을까,스포츠,84
3004641,1일 경상북도 칠곡군 왜관에 위치한 파미힐스CC에서 열린 2017 DGB금융그룹 대...,포즈취하는 정한밀,스포츠,85


In [ ]:
df.to_pickle('newsis_sec.pkl')

## 섹션 정리

In [5]:
df = pd.read_pickle('newsis_sec.pkl')

In [6]:
counts=df.groupby('sec').size()
sorted_counts=counts.sort_values(ascending=False)
sorted_counts

sec
사회      911719
전국      573195
정치      540389
경제      305404
스포츠     275750
국제      220694
연예       77112
문화       56073
산업       33739
사람들       5277
광장        2784
기타        1113
라이프       1035
오피니언       345
국감          13
dtype: int64

In [7]:
df[df['sec']=='산업']

,article,title,sec,len
598807,프랜차이즈 커피전문점은 신임 대표이사에 유재면(사진) 전 웅진식품 대표를 선임했다고...,"드롭탑, 신임 대표이사에 유재면 前 웅진식품 대표",산업,334
1327124,우영윤씨가 서울 양평동 자신의 사무실에서 고객과 상담을 하고 있다. 해외 유학을...,"[청년, 창업에서 희망을 보다④]“사업도 창작…해외여행하며 영감얻어”",산업,2175
1447340,커피전문점 드롭탑은 김택(사진) 대표이사를 새로운 최고경영자(CEO)로영입했다고 2...,"드롭탑, 김택 신임 대표 영입…해외공략 박차",산업,380
2373206,아우디는 5일(현지시간) 미국 라스베이거스에서 열리는 'CES 2017'에서 엔비디...,"[CES2017]아우디, AI 접목한 자율주행기술 선보여",산업,1103
2373207,폭스바겐은 미국 라스베이거스에서 열리는 'CES 2017'에서 고객들과 자동차와의 ...,"[CES2017]폭스바겐, 자율주행콘셉트카 'I.D.' 공개",산업,570
2373223,장세영 기자= 지난해 영업 종료 이후 193일 만의 영업 재개한 서울 송파구 롯데면...,롯데면세점 월드타워점 재개장,산업,210
2373291,"▲쌤소나이트 레드가 한국과 일본, 중국 등지에서 글로벌 인기를 얻고 있는 배우 이종...","[패션단신]쌤쏘나이트 레드, 배우 이종석 모델 발탁 外",산업,511
2373307,삼성전자 윤부근 대표이사 사장이 4일(현지시각) 미국 라스베이거스에서 'CES 20...,기자간담회 하는 윤부근 대표이사 사장,산업,212
2373309,삼성전자 윤부근 대표이사 사장이 4일(현지시각) 미국 라스베이거스에서 'CES 20...,기자간담회 하는 윤부근 대표이사 사장,산업,212
2373310,삼성전자 윤부근 대표이사 사장이 4일(현지시각) 미국 라스베이거스에서 'CES 20...,간담회 하는 윤부근,산업,212


In [4]:
counts=df.groupby('sec').size()
section_idx=enumerate(counts.keys())
section_to_int = {word: ii for ii, word in section_idx}
section_to_int

{'경제': 0,
 '광장': 1,
 '국감': 2,
 '국제': 3,
 '기타': 4,
 '라이프': 5,
 '문화': 6,
 '사람들': 7,
 '사회': 8,
 '산업': 9,
 '스포츠': 10,
 '연예': 11,
 '오피니언': 12,
 '전국': 13,
 '정치': 14}

In [ ]:
def set_section_idx(section):
    return section_to_int[section]

In [ ]:
pickle_dict1['sec_idx']=pickle_dict1['sec'].apply(set_section_idx)


## 문장추출

In [3]:
doc_sentences=df['article'].str.findall(r"[0-9a-zA-Z가-힣%,()\- ]+\.")

In [4]:
len(doc_sentences)

3004642

In [5]:
%%time
from konlpy.tag import Mecab; m = Mecab()
pos = lambda d: ['/'.join(p) for p in m.pos(d)]

tagged_sentences=doc_sentences.apply(lambda x:pos(''.join(x)))

KeyboardInterrupt: 

In [6]:
words_count=tagged_sentences.apply(lambda x:len(x))

In [7]:
df['sentences']=doc_sentences
df['tagged_sentences']=tagged_sentences
df['words_count']=words_count

In [10]:
df[df['words_count']==0]

,article,title,sec,len,sec_idx,sentences,tagged_sentences,words_count
1,"2일부터 오는 6일까지 계룡대에서 개최되는 '2013년 계룡 군문화축제, 지상군페스...",6.25전쟁 사진전 보는 학생들,사회,123,8,[],[],0
2,1일 오전 서울 동대문구청사에서 열린 학교급식 식재료의 질적 개선과 안정적인 공급을...,'어느 쌀이 우수할까',사회,89,8,[],[],0
7,김석균(왼쪽) 해양경찰청장이 1일 오후 서해·제주 해역에서 광역초계기(CN-235)...,"김석균 해양경찰청장, 외국어선 조업실태 점검",사회,81,8,[],[],0
13,1일 오후 경기도 고양의 어울림누리 빙상장에서 열린 ‘2013-2014 아시아리그 ...,센스 있는 심판!,스포츠,97,10,[],[],0
15,2일 오후 서울 마포구 공덕동 신용보증기금에서 열린 신용보증기금 서근우 신임 이사장...,취임사하는 서근우 신임 이사장,경제,73,0,[],[],0
17,2일 오후 서울 마포구 공덕동 신용보증기금에서 열린 신용보증기금 서근우 신임 이사장...,사기 흔드는 서근우 이사장,경제,73,0,[],[],0
18,2일 오후 서울 마포구 공덕동 신용보증기금에서 열린 신용보증기금 서근우 신임 이사장...,취임사하는 서근우 신임 이사장,경제,73,0,[],[],0
25,2일 오후 서울 잠실야구장에서 열린 2013 한국야쿠르트 세븐 프로야구 한화 이글스...,역투하는 한화 이브랜드,스포츠,81,10,[],[],0
26,2일 오후 서울 잠실야구장에서 열린 2013 한국야쿠르트 세븐 프로야구 한화 이글스...,선취점 얻어내는 LG,스포츠,126,10,[],[],0
27,2일 오후 서울 잠실야구장에서 열린 2013 한국야쿠르트 세븐 프로야구 한화 이글스...,"LG 김기태 감독, '좋았어'",스포츠,127,10,[],[],0


In [28]:
df.iloc[1,0]

"2일부터 오는 6일까지 계룡대에서 개최되는 '2013년 계룡 군문화축제, 지상군페스티벌'에서 (사)월드피스자유연합 (대표 안재철)이 6.25전쟁 사진전과 6.25전쟁 당시 대한민국을 지원한 67개국 국기를 전시하고 있다."

In [26]:
df[df.index==1].article

1    2일부터 오는 6일까지 계룡대에서 개최되는 '2013년 계룡 군문화축제, 지상군페스...
Name: article, dtype: object

In [47]:
df.reset_index(level=0, inplace=True)

In [49]:
df.drop(['index'], axis = 1, inplace = True)

In [51]:
df.to_pickle('kbo_parsed.pkl')

In [90]:
df['title']=df['title'].str.replace(r"[^0-9a-zA-Z가-힣,.!?@=<>()\[\]{} ]+", " ")
df['contents']=df['contents'].str.replace(r"[^0-9a-zA-Z가-힣,.!?@=<>()\[\]{} ]+|[0-9a-zA-Z.]+.kr|[0-9a-zA-Z.]+.com|[0-9a-zA-Z.]+@[a-zA-Z.]+|[0-9a-zA-Z.]+@|\([a-z.:/]+\)|마이데일리.|조이뉴스24.|[a-zA-Z,. ]+\.|기사제공 [0-9a-zA-Z가-힣 .]", " ")

In [96]:
df['contents']=df['contents'].str.findall(r"[0-9a-zA-Z가-힣,.!?<>() ]+\. ").apply(lambda x: ' '.join(x))